# Audio-Entrauschung mit einem Denoising Autoencoder

In diesem Notebook bauen wir mit PyTorch einen Denoising Autoencoder, um Rauschen aus Audiodaten zu entfernen.

Wir verwenden den **YESNO**-Datensatz aus dem `torchaudio`-Paket, der aus Aufnahmen einer Person besteht, die auf Hebräisch "ja" oder "nein" sagt. Wir fügen diesen Aufnahmen künstlich Rauschen hinzu und trainieren einen Autoencoder, um die ursprünglichen sauberen Audiosignale aus den verrauschten Eingaben zu rekonstruieren.

**Schritte:**

1. **Import der notwendigen Bibliotheken**: Wir verwenden PyTorch und torchaudio für den Modellaufbau und die Datenverarbeitung.
2. **Vorbereitung des Datensatzes**: Laden des YESNO-Datensatzes und Erstellen eines benutzerdefinierten Datensatzes, der den Audiodateien Rauschen hinzufügt.
3. **Definition des Autoencoder-Modells**: Aufbau eines einfachen vollständig verbundenen Autoencoders, der für die Audiodaten geeignet ist.
4. **Training des Modells**: Einrichten der Trainingsschleife, um den Rekonstruktionsverlust zwischen der entrauschten Ausgabe und dem ursprünglichen sauberen Audio zu minimieren.
5. **Speichern des trainierten Modells**: Nach dem Training speichern wir den Zustand des Modells für die spätere Verwendung.

(Dieses Notebook wurde mithilfe von GPT o1-preview erstellt, per Spracheingabe spontan im Laufen: [Chatverlauf](https://chatgpt.com/share/66e368a5-10ec-800e-a136-e5e48456fb25))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchaudio

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 50
batch_size = 16
learning_rate = 0.001

# Load the YESNO dataset
dataset = torchaudio.datasets.YESNO(root='.', download=True)

# Custom dataset to add noise to the clean audio
class NoisyYesNoDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.noise_factor = 0.1
        self.fixed_length = 8000  # 1 second at 8000 Hz

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, sample_rate, labels = self.dataset[idx]
        # Normalize waveform
        waveform = waveform / waveform.abs().max()
        # Pad or truncate waveform to fixed length
        if waveform.size(1) < self.fixed_length:
            pad_size = self.fixed_length - waveform.size(1)
            waveform = torch.nn.functional.pad(waveform, (0, pad_size))
        else:
            waveform = waveform[:, :self.fixed_length]
        # Add noise
        noise = self.noise_factor * torch.randn(waveform.size())
        noisy_waveform = waveform + noise
        # Ensure the noisy waveform is still in the same range
        noisy_waveform = torch.clamp(noisy_waveform, -1.0, 1.0)
        return noisy_waveform, waveform

# Create dataset and dataloader
noisy_dataset = NoisyYesNoDataset(dataset)
dataloader = DataLoader(noisy_dataset, batch_size=batch_size, shuffle=True)

# Define the Autoencoder model
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(8000, 4000),
            nn.ReLU(),
            nn.Linear(4000, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500)
        )
        self.decoder = nn.Sequential(
            nn.Linear(500, 1000),
            nn.ReLU(),
            nn.Linear(1000, 4000),
            nn.ReLU(),
            nn.Linear(4000, 8000),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize model, loss function and optimizer
model = DenoisingAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for batch_idx, (noisy_waveforms, clean_waveforms) in enumerate(dataloader):
        # Flatten waveforms
        noisy_waveforms = noisy_waveforms.squeeze(1).to(device)
        clean_waveforms = clean_waveforms.squeeze(1).to(device)
        # Forward pass
        outputs = model(noisy_waveforms)
        # Compute loss
        loss = criterion(outputs, clean_waveforms)
        # Zero gradients
        optimizer.zero_grad()
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'denoising_autoencoder.pth')


Epoch [1/50], Loss: 0.0068
Epoch [2/50], Loss: 0.0100
Epoch [3/50], Loss: 0.0118
Epoch [4/50], Loss: 0.0053
Epoch [5/50], Loss: 0.0082
Epoch [6/50], Loss: 0.0107
Epoch [7/50], Loss: 0.0055
Epoch [8/50], Loss: 0.0106
Epoch [9/50], Loss: 0.0055
Epoch [10/50], Loss: 0.0047
Epoch [11/50], Loss: 0.0051
Epoch [12/50], Loss: 0.0073
Epoch [13/50], Loss: 0.0068
Epoch [14/50], Loss: 0.0024
Epoch [15/50], Loss: 0.0036
Epoch [16/50], Loss: 0.0037
Epoch [17/50], Loss: 0.0027
Epoch [18/50], Loss: 0.0036
Epoch [19/50], Loss: 0.0035
Epoch [20/50], Loss: 0.0026
Epoch [21/50], Loss: 0.0020
Epoch [22/50], Loss: 0.0026
Epoch [23/50], Loss: 0.0019
Epoch [24/50], Loss: 0.0012
Epoch [25/50], Loss: 0.0019
Epoch [26/50], Loss: 0.0009
Epoch [27/50], Loss: 0.0011
Epoch [28/50], Loss: 0.0010
Epoch [29/50], Loss: 0.0013
Epoch [30/50], Loss: 0.0011
Epoch [31/50], Loss: 0.0011
Epoch [32/50], Loss: 0.0005
Epoch [33/50], Loss: 0.0009
Epoch [34/50], Loss: 0.0006
Epoch [35/50], Loss: 0.0007
Epoch [36/50], Loss: 0.0006
E

## Demonstration der Entrauschungsfähigkeit des Modells

Nachdem wir unseren Denoising Autoencoder trainiert haben, testen wir nun seine Leistung an einem Beispiel aus dem Datensatz. Wir werden:

- Eine zufällige Audiospur aus dem Datensatz auswählen.
- Das originale saubere Audio, die verrauschte Version und die entrauschte Ausgabe unseres Modells anzeigen.

Dies hilft uns, die Effektivität unseres Modells bei der Entfernung von Rauschen aus Audiodaten zu testen.


In [ ]:
import matplotlib.pyplot as plt
import IPython.display as ipd
import random

# Funktion zum Anzeigen der Kapazität des Modells
def display_audio_example(model, dataset, device):
    # Set the model to evaluation mode
    model.eval()
    # Choose a random sample from the dataset
    index = random.randint(0, len(dataset) - 1)
    noisy_waveform, clean_waveform = dataset[index]

    # Prepare tensors for the model
    noisy_waveform = noisy_waveform.squeeze(0).to(device).unsqueeze(0)  # Add batch dimension
    clean_waveform = clean_waveform.squeeze(0).to(device).unsqueeze(0)  # Add batch dimension

    # Predict the denoised output
    with torch.no_grad():
        denoised_waveform = model(noisy_waveform).cpu().squeeze(0)  # Remove batch dimension

    # Display the audio and the plots
    print("Original Clean Audio:")
    ipd.display(ipd.Audio(clean_waveform.cpu().numpy(), rate=8000))

    print("Noisy Audio:")
    ipd.display(ipd.Audio(noisy_waveform.cpu().squeeze(0).numpy(), rate=8000))

    print("Denoised Audio:")
    ipd.display(ipd.Audio(denoised_waveform.numpy(), rate=8000))

# Display an example
display_audio_example(model, noisy_dataset, device)


Original Clean Audio:


Noisy Audio:


Denoised Audio:


## Aufgabe
Die Entrauschung ist noch nicht sehr gut.
Optimieren Sie die Audio-Entrauschung so, dass der Output weniger Rauschen zeigt als der Input, die Sprache aber trotzdem verständlich bleibt.